In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
data = pd.read_csv("dataset/mbti_1.csv")

In [38]:
data.head(16)

,type,posts,type_index,cleaned_text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,enfp and intj moments sportscenter not t...
1,ENTP,'I'm finding the lack of me in these posts ver...,3,im finding the lack of me in these pos...
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,good one of course to which i say i ...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,dear intp i enjoyed our conversation the...
4,ENTJ,'You're fired.|||That's another silly misconce...,2,youre fired thats another silly misconcep...
5,INTJ,'18/37 @.@|||Science is not perfect. No scien...,10,1837 science is not perfect no scienti...
6,INFJ,"'No, I can't draw on my own nails (haha). Thos...",8,no i cant draw on my own nails haha t...
7,INTJ,'I tend to build up a collection of things on ...,10,i tend to build up a collection of thi...
8,INFJ,"I'm not sure, that's a good question. The dist...",8,im not sure thats a good question the ...
9,INTP,'https://www.youtube.com/watch?v=w8-egj0y8Qs||...,11,im in this position where i have to ac...


In [4]:
types = np.unique(data.type.values)

In [5]:
def get_type_index(string):
    return list(types).index(string)
data['type_index'] = data['type'].apply(get_type_index)

In [6]:
data.posts.values[0]

"'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...

In [7]:
import string
import re

def clean_text(text):
    regex = re.compile('[%s]' % re.escape('|'))
    text = regex.sub(" ", text)
    words = str(text).split()
    words = [i.lower() + " " for i in words]
    words = [i for i in words if not "http" in i]
    words = " ".join(words)
    words = words.translate(words.maketrans('', '', string.punctuation))
    return words

In [8]:
data['cleaned_text'] = data['posts'].apply(clean_text)

In [9]:
data.cleaned_text.values[0]

'enfp  and  intj  moments  sportscenter  not  top  ten  plays  pranks  what  has  been  the  most  lifechanging  experience  in  your  life  on  repeat  for  most  of  today  may  the  perc  experience  immerse  you  the  last  thing  my  infj  friend  posted  on  his  facebook  before  committing  suicide  the  next  day  rest  in  peace  hello  enfj7  sorry  to  hear  of  your  distress  its  only  natural  for  a  relationship  to  not  be  perfection  all  the  time  in  every  moment  of  existence  try  to  figure  the  hard  times  as  times  of  growth  as  84389  84390    welcome  and  stuff  game  set  match  prozac  wellbrutin  at  least  thirty  minutes  of  moving  your  legs  and  i  dont  mean  moving  them  while  sitting  in  your  same  desk  chair  weed  in  moderation  maybe  try  edibles  as  a  healthier  alternative  basically  come  up  with  three  items  youve  determined  that  each  type  or  whichever  types  you  want  to  do  would  more  than  likely  us

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data)
train, val = train_test_split(train)


In [55]:
print (train.shape)
print (val.shape)
print (test.shape)

(4879, 4)
(1627, 4)
(2169, 4)


In [17]:
!pip install transformers

     |████████████████████████████████| 1.3MB 3.5MB/s 
     |████████████████████████████████| 890kB 13.9MB/s 
     |████████████████████████████████| 2.9MB 23.6MB/s 
     |████████████████████████████████| 1.1MB 47.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3312a3bc5115c62e754b8ae810e3a9faf7ed9b4e6b14a67db4692251b263abd0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [22]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-large-uncased')

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten, Dropout, Conv1D, GlobalMaxPooling1D

def create_model():
    op = tf.keras.optimizers.Adam(learning_rate=0.00001)

    model = Sequential()
    model.add(Embedding(vocab_size, 256, input_length=maxlen-1))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(200, return_sequences=True)))
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(20)))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(16, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=op, metrics=['accuracy'])
    return model

In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
trunc_type = "post"
pad_type = "post"
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(data.cleaned_text.values)

In [57]:
maxlen = 1500
train_sequences = tokenizer.texts_to_sequences(train.cleaned_text.values)
train_padded = pad_sequences(train_sequences, maxlen = maxlen, truncating = trunc_type, padding = pad_type)

val_sequences = tokenizer.texts_to_sequences(val.cleaned_text.values)
val_padded = pad_sequences(val_sequences, maxlen = maxlen, truncating = trunc_type, padding = pad_type)

In [58]:
print (train_padded.shape)
print (val_padded.shape)

(4879, 1500)
(1627, 1500)


In [ ]:
use_tpu = False
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()
    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1499, 256)         2560000   
_________________________________________________________________
dropout_146 (Dropout)        (None, 1499, 256)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 1499, 400)         731200    
_________________________________________________________________
dropout_147 (Dropout)        (None, 1499, 400)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40)                67360     
_________________________________________________________________
dropout_148 (Dropout)        (None, 40)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2

In [51]:
one_hot_labels = tf.keras.utils.to_categorical(train.type_index.values, num_classes=16)
val_labels= tf.keras.utils.to_categorical(val.type_index.values, num_classes=16)

In [54]:
one_hot_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model.fit(train_padded, one_hot_labels, epochs =20, verbose = 1, 
          validation_data = (val_padded, val_labels),  callbacks = [tf.keras.callbacks.EarlyStopping(patience = 3)])

Epoch 1/20


  2/153 [..............................] - ETA: 2:51 - loss: 2.7706 - accuracy: 0.0938WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0014s vs `on_train_batch_end` time: 2.2603s). Check your callbacks.


153/153 [==============================] - ETA: 0s - loss: 2.7562 - accuracy: 0.1209WARNING:tensorflow:Model was constructed with shape (None, 1499) for input Tensor("embedding_input:0", shape=(None, 1499), dtype=float32), but it was called on an input with incompatible shape (None, 1500).


153/153 [==============================] - 377s 2s/step - loss: 2.7562 - accuracy: 0.1209 - val_loss: 2.7320 - val_accuracy: 0.1309
Epoch 2/20
153/153 [==============================] - 378s 2s/step - loss: 2.7045 - accuracy: 0.1301 - val_loss: 2.6568 - val_accuracy: 0.1266
Epoch 3/20
153/153 [==============================] - 382s 2s/step - loss: 2.6159 - accuracy: 0.1289 - val_loss: 2.5366 - val_accuracy: 0.1266
Epoch 4/20
153/153 [==============================] - 383s 3s/step - loss: 2.5340 - accuracy: 0.1287 - val_loss: 2.4307 - val_accuracy: 0.1266
Epoch 5/20
153/153 [==============================] - 384s 3s/step - loss: 2.4562 - accuracy: 0.1685 - val_loss: 2.3695 - val_accuracy: 0.2090
Epoch 6/20
153/153 [==============================] - 387s 3s/step - loss: 2.4191 - accuracy: 0.1781 - val_loss: 2.3453 - val_accuracy: 0.2090
Epoch 7/20
153/153 [==============================] - 386s 3s/step - loss: 2.3962 - accuracy: 0.1841 - val_loss: 2.3307 - val_accuracy: 0.2090
Epoch 8/20

In [ ]:
model.save("models/personality_model.h5")

In [13]:
new_model = tf.keras.models.load_model('models/personality_model.h5')

# Check its architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1499, 256)         2560000   
_________________________________________________________________
dropout_146 (Dropout)        (None, 1499, 256)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 1499, 400)         731200    
_________________________________________________________________
dropout_147 (Dropout)        (None, 1499, 400)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 40)                67360     
_________________________________________________________________
dropout_148 (Dropout)        (None, 40)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2

In [27]:
test_labels= tf.keras.utils.to_categorical(test.type_index.values, num_classes=16)

In [30]:
test_sequences = tokenizer.texts_to_sequences(test.cleaned_text.values)
test_padded = pad_sequences(test_sequences, maxlen = 1500, truncating = trunc_type, padding = pad_type)

In [31]:
new_model.evaluate(np.array(test_padded), test_labels)

68/68 [==============================] - 203s 3s/step - loss: 2.2858 - accuracy: 0.2112


[2.2857666015625, 0.21115721762180328]

In [32]:
print(new_model.predict(test_padded))

[[0.01755055 0.0746889  0.03881089 ... 0.02421712 0.03078783 0.04636303]
 [0.01594081 0.07214715 0.03871734 ... 0.02321349 0.02850211 0.04422106]
 [0.01591561 0.07223146 0.03868169 ... 0.02324132 0.02850406 0.04427687]
 ...
 [0.01594858 0.07234409 0.03864701 ... 0.02324031 0.02856634 0.0442911 ]
 [0.01590267 0.07214415 0.03858009 ... 0.02319117 0.02843972 0.04423463]
 [0.01759198 0.07485618 0.03879917 ... 0.02430656 0.03092699 0.04642294]]


In [70]:
input_text = "enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks"
cleaned_ip = clean_text(input_text)
list_input = []
list_input.append(input_text)
custom_test_sequences = tokenizer.texts_to_sequences(list_input)
print (custom_test_sequences)
custom_test_padded = pad_sequences(custom_test_sequences, maxlen = maxlen, truncating = trunc_type, padding = pad_type)

[[190, 6, 139, 1305, 1, 1, 916, 1, 487, 2763, 1, 1, 23, 704, 1856, 2079, 1, 1, 916, 1, 487, 2763, 1, 9953]]


In [71]:
print (custom_test_padded.shape)

(1, 1500)


In [76]:
type_person = np.argmax(new_model.predict(custom_test_padded))
print (data[data["type_index"]==type_person]["type"].head(1))

17    INFP
Name: type, dtype: object
